**注意事項**

このノートブックは、GPU:「T4」に対応させたものです。
「L4」版のノートブックとはモデル等が異なるため、生成される内容が異なることが考えられます。

生成される内容と、ノートブックに記載されている説明が一致しない場合があることをご了承ください。

生成内容とノートブックの説明をよく見比べ、適宜読み替えながら演習を進めてみてください。

---

# 演習の方針

1. **ベースラインモデル評価**  
   素のモデルで回答を生成し、講義内容との整合性の低さを観察します。これにより、特別な学習なしでのモデルの限界を確認します。

2. **文字起こしデータの活用**  
   講義の文字起こしデータを導入し、モデルが講義内容を参照した回答を生成する傾向を観察します。ただし、Retrieval（情報検索）精度の限界から結果は不安定になる可能性があります。

3. **チャンク化の導入**  
   文字起こしデータをチャンク（小単位）に分割し、より安定して関連コンテンツを取得できるようにします。この段階では文脈理解にまだ課題があることを確認します。

4. **Rerankの適用**  
   検索結果のランク付けを導入し、より的確で安定した回答を目指します。

5. **応用改善手法**  
   文字起こしの品質向上のための編集技術や、メタデータの活用による性能向上手法を探ります。

## 扱う質問

「Inference Time Scaling（推論時スケーリング）」に関する質問を取り扱います。これは以下の背景を持つトピックです。

- 2024年8月発表の論文「Scaling LLM Test-Time Compute Optimally can be More Effective than Scaling Model Parameters」で提唱された概念
- OpenAIのGPT-o1（2024年9月リリース）で実用化され、注目を集めた比較的新しいアプローチ
- 2024年度LLM講座の第4回講義でも取り上げられた重要テーマ

## 扱うモデル

「google/gemma-2-2b-jpn-it」を使用します。このモデルは、リリース時期の関係上、以下の特徴を持ちます。

- 「Inference Time Scaling」の概念が広まる前に訓練されており、このトピックに関する知識を持たないと想定される
- この特性を活かし、純粋なベースライン評価から各手法の効果を観察する

### 演習環境の準備

In [ ]:
!pip install --upgrade transformers
!pip install google-colab-selenium
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 66.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 115.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# 演習用のコンテンツを取得
!git clone https://github.com/tshigata/lecture-ai-engineering.git

Cloning into 'lecture-ai-engineering'...
remote: Enumerating objects: 149, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 149 (delta 8), reused 0 (delta 0), pack-reused 133 (from 4)
Receiving objects: 100% (149/149), 231.68 KiB | 1.50 MiB/s, done.
Resolving deltas: 100% (57/57), done.


In [ ]:
from google.colab import userdata
hf_token = userdata.get('HF_TOKEN')

# HuggingFace Login
from huggingface_hub import notebook_login
from huggingface_hub import login

login(token=hf_token)

In [ ]:
# CUDAが利用可能ならGPUを、それ以外ならCPUをデバイスとして設定
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
import random
random.seed(0)

In [ ]:
# モデル(Gemma2)の読み込み

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = "google/gemma-2-2b-jpn-it"
tokenizer = AutoTokenizer.from_pretrained(model_name)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
            model_name,
            device_map="auto",
            quantization_config=bnb_config,
            torch_dtype=torch.bfloat16,
        )

tokenizer_config.json:   0%|          | 0.00/46.9k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/555 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [ ]:
# 質問リストを定義
questions = [
    "松尾・岩澤研究室の講座において、退会した後も研究室が受講者の情報を利用できる条件とは何ですか？",
    "未成年者が講座を受講するには、どのような条件が求められますか？",
    "受講者が他の人のSlack投稿をSNSで共有することは、講座規約上どう扱われますか？",
    "講座で使用された資料や動画を、受講後に再利用・再配布することは許可されていますか？",
    "講義中にノイズなどのトラブルを発生させた場合、講座への参加はどうなりますか？"
]

In [ ]:
for question in questions:
  print(question)


松尾・岩澤研究室の講座において、退会した後も研究室が受講者の情報を利用できる条件とは何ですか？
未成年者が講座を受講するには、どのような条件が求められますか？
受講者が他の人のSlack投稿をSNSで共有することは、講座規約上どう扱われますか？
講座で使用された資料や動画を、受講後に再利用・再配布することは許可されていますか？
講義中にノイズなどのトラブルを発生させた場合、講座への参加はどうなりますか？


# 1. ベースラインモデル評価
**まずはベースモデルがどの程度知識を持っているか確かめる**

In [ ]:
def llm_answer(question):
    messages = [
        {"role": "user", "content": question},
    ]

    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = model.generate(
        input_ids,
        max_new_tokens=256,
        eos_token_id=terminators,
        do_sample=False,
    )

    response = outputs[0][input_ids.shape[-1]:]

    return tokenizer.decode(response, skip_special_tokens=True)

In [ ]:
# ベースライン回答の生成（RAGなし）
baseline_answers = [llm_answer(q) for q in questions]
for answer in baseline_answers:
  print(answer)

 松尾・岩澤研究室の講座において、退会後も研究室が受講者の情報を利用できる条件は、**研究室のポリシーや契約内容によって異なります。** 

一般的には、以下の様な状況が考えられます。

* **研究室のポリシー**:  研究室が受講者の情報を利用するかどうかを明確に定めたポリシーがある場合、そのポリシーに従います。
* **契約内容**:  講座の参加に際して、受講者と研究室が契約を締結した場合は、契約内容に記載されている内容に従います。
* **個人情報保護**:  研究室は、受講者の個人情報保護に配慮し、利用する場合は、法令や倫理的なガイドラインに従います。


**具体的な情報を得るためには、以下の方法をお勧めします。**

* **研究室のウェブサイト**:  研究室のウェブサイトで、講座に関する情報やポリシーについて確認できます。
* **研究室の担当者**:  直接研究室の担当者に問い合わせて、詳細な情報を得ることができます。
* **講座の案内**:  講座の案内資料に、退会後の利用状況に関する情報が記載されている場合があります。



 



未成年者が講座を受講するには、以下の条件が求められます。

**1.  年齢制限:**
   *  講座によって年齢制限は異なりますが、一般的には18歳未満の未成年者向け講座は、保護者または親権者の同意が必要です。

**2.  保護者または親権者の同意:**
   *  講座の内容によっては、保護者または親権者の同意が必要となる場合があります。
   *  同意書が必要となる場合もあります。

**3.  講座の内容:**
   *  講座の内容によっては、年齢制限や保護者同意が必要となる場合があります。
   *  例えば、性教育や医療に関する講座は、保護者または親権者の同意が必要となる場合が多いです。

**4.  講座の運営機関:**
   *  講座の運営機関によっては、未成年者向けの講座を運営する際に、年齢制限や保護者同意の確認が必要となる場合があります。


**注意点:**

*  講座の内容によっては、年齢制限や保護者同意が必要となる場合があります。
*  未成年者向けの講座を受講する際は、必ず運営機関のウェブサイトや連絡先を確認してください。



 



講座規約上、受講者が他の人の

In [ ]:
from sentence_transformers import SentenceTransformer

emb_model = SentenceTransformer("infly/inf-retriever-v1-1.5b", trust_remote_code=True)
# In case you want to reduce the maximum length:
emb_model.max_seq_length = 4096

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/284 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/19.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/55.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/918 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/370 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

# 2. RAGモデル評価

In [ ]:
with open("/content/lecture-ai-engineering/day3/data/松尾・岩澤研究室 講座受講規約.txt", "r", encoding="shift_jis") as f:
    raw_writedown = f.read()

In [ ]:
# ドキュメントを用意する。
documents = [text.strip() for text in raw_writedown.split("。")]
print("ドキュメントサイズ: ", len(documents))
print("ドキュメントの例: \n", documents[0:2])

ドキュメントサイズ:  62
ドキュメントの例: 
 ['松尾・岩澤研究室 講座受講規約\nこの講座登録規約（以下「本規約」）は、東京大学大学院 工学系研究科 技術経営戦略学専攻 松尾・岩澤究室（以下「当研究室」）が提供する講座、またそれを含むシステム等（以下「本サービス」）の利用する際に適用する事項を定めるものです', '第1条（適用範囲）\n1. 本規約は、講座を受講するすべての対象者(以下、受講者)と当研究室との間における一切の関係に適用されるものとします']


In [ ]:
import torch

# GPUのメモリを解放
torch.cuda.empty_cache() #-> NG

# # CPUに切り替え
emb_model = emb_model.to("cpu")

In [ ]:
# 参照テキストのembeddingを生成する
doc_embeddings = emb_model.encode(documents, convert_to_tensor=True)

In [ ]:
def get_references_from_question(question, topk=5):
    q_embedding = emb_model.encode([question], convert_to_tensor=True)
    scores = torch.matmul(q_embedding, doc_embeddings.T)[0].cpu().numpy()
    top_indices = scores.argsort()[::-1][:topk]
    return "\n".join([f"* {documents[i]}" for i in top_indices])

In [ ]:
rag_prompts = [
    f"以下の講座規約を参考にして質問に答えてください。\n\n{get_references_from_question(q)}\n\n質問：{q}"
    for q in questions
]

print(rag_prompts)

['以下の講座規約を参考にして質問に答えてください。\n\n* 4. 退会を希望する受講者は、退会後においても本サービス利用中に当研究室が取得した受講者情報を別途定めるプライバシーポリシーを遵守しながら、当研究室が本規約に従って利用できることに同意します\n* 2. 受講者が各講座の受講を修了していたとしても受講者登録が行われている場合、当研究室から各受講者に新たな講座案内等の情報発信を行うことがあります\n* 第7条（退会）\n1. 受講者は、当研究室の定める退会手続により、本サービスから退会できるものとします\n* 松尾・岩澤研究室 講座受講規約\nこの講座登録規約（以下「本規約」）は、東京大学大学院 工学系研究科 技術経営戦略学専攻 松尾・岩澤究室（以下「当研究室」）が提供する講座、またそれを含むシステム等（以下「本サービス」）の利用する際に適用する事項を定めるものです\n* 2. なお、本規約より受講者は受講者からの質問や個人情報に該当しない部分については研究室がデータの利活用する事を了承するものとします\n\n質問：松尾・岩澤研究室の講座において、退会した後も研究室が受講者の情報を利用できる条件とは何ですか？', '以下の講座規約を参考にして質問に答えてください。\n\n* 4. 受講希望者が未成年者、成年被後見人、被保佐人または被補助人のいずれかの場合、本人が申込むことができますが、事前に法定代理人、後見人、保佐人または補助人の同意を得ていることが必要です\n* 2. 各講座は学生の有無、属性等を基準に受講資格が設定されています\n* 第2条（受講資格）\n1. 受講者は、講座に登録するために当研究室が発行するアカウントIDが、講座申込には必要です\n* それらを満たし修了生として相応しいと評価された場合に講座修了とみなします\n* 3. 各講座にて成績評価・修了要件を提示いたします\n\n質問：未成年者が講座を受講するには、どのような条件が求められますか？', '以下の講座規約を参考にして質問に答えてください。\n\n* 2. なお、本規約より受講者は受講者からの質問や個人情報に該当しない部分については研究室がデータの利活用する事を了承するものとします\n* 第1条（適用範囲）\n1. 本規約は、講座を受講するすべての対象者(以下、受講者)と当研究室と

In [ ]:
rag_answers = []
for i, q in enumerate(questions):
  rag_prompt = f"以下の講座規約を参考にして質問に答えてください。\n\n{get_references_from_question(q)}\n\n質問：{q}"
  rag_response = llm_answer(rag_prompt)
  rag_answers.append(rag_response)

In [ ]:
# prompt: # questions, baseline_answers, rag_answers を一つのDataframeに収める

import pandas as pd

# Assuming questions, baseline_answers, and rag_answers are already defined as in your provided code

df = pd.DataFrame({
    'questions': questions,
    'baseline_answers': baseline_answers,
    'rag_answers': rag_answers
})

df


,questions,baseline_answers,rag_answers
0,松尾・岩澤研究室の講座において、退会した後も研究室が受講者の情報を利用できる条件とは何ですか？,松尾・岩澤研究室の講座において、退会後も研究室が受講者の情報を利用できる条件は、**研究室...,松尾・岩澤研究室の講座において、退会した後も研究室が受講者の情報を利用できる条件は、**「本...
1,未成年者が講座を受講するには、どのような条件が求められますか？,未成年者が講座を受講するには、以下の条件が求められます。\n\n**1. 年齢制限:**\...,未成年者が講座を受講するには、以下の条件が求められます。 \n\n* **未成年者の本人によ...
2,受講者が他の人のSlack投稿をSNSで共有することは、講座規約上どう扱われますか？,講座規約上、受講者が他の人のSlack投稿をSNSで共有することは、**多様で、明確な回答は...,講座規約に記載されている通り、**受講者が他の人のSlack投稿をSNSで共有することは、個...
3,講座で使用された資料や動画を、受講後に再利用・再配布することは許可されていますか？,講座で使用された資料や動画の再利用・再配布については、**講座の主催者や提供者によって異なり...,いいえ、講座で使用された資料や動画を、受講後に再利用・再配布することは許可されていません。 ...
4,講義中にノイズなどのトラブルを発生させた場合、講座への参加はどうなりますか？,講義中にノイズなどのトラブルが発生した場合、講座への参加は**状況によって異なります**。\...,講義中にノイズなどのトラブルが発生した場合、当研究室の判断により、受講環境が改善するまでの間...


In [ ]:
df.to_csv("llm_responses.csv", index=False, encoding="utf-8")

In [ ]:
baseline_answers

[' 松尾・岩澤研究室の講座において、退会後も研究室が受講者の情報を利用できる条件は、**研究室のポリシーや契約内容によって異なります。** \n\n一般的には、以下の様な状況が考えられます。\n\n* **研究室のポリシー**:  研究室が受講者の情報を利用するかどうかを明確に定めたポリシーがある場合、そのポリシーに従います。\n* **契約内容**:  講座の参加に際して、受講者と研究室が契約を締結した場合は、契約内容に記載されている内容に従います。\n* **個人情報保護**:  研究室は、受講者の個人情報保護に配慮し、利用する場合は、法令や倫理的なガイドラインに従います。\n\n\n**具体的な情報を得るためには、以下の方法をお勧めします。**\n\n* **研究室のウェブサイト**:  研究室のウェブサイトで、講座に関する情報やポリシーについて確認できます。\n* **研究室の担当者**:  直接研究室の担当者に問い合わせて、詳細な情報を得ることができます。\n* **講座の案内**:  講座の案内資料に、退会後の利用状況に関する情報が記載されている場合があります。\n\n\n\n \n\n\n',
 '未成年者が講座を受講するには、以下の条件が求められます。\n\n**1.  年齢制限:**\n   *  講座によって年齢制限は異なりますが、一般的には18歳未満の未成年者向け講座は、保護者または親権者の同意が必要です。\n\n**2.  保護者または親権者の同意:**\n   *  講座の内容によっては、保護者または親権者の同意が必要となる場合があります。\n   *  同意書が必要となる場合もあります。\n\n**3.  講座の内容:**\n   *  講座の内容によっては、年齢制限や保護者同意が必要となる場合があります。\n   *  例えば、性教育や医療に関する講座は、保護者または親権者の同意が必要となる場合が多いです。\n\n**4.  講座の運営機関:**\n   *  講座の運営機関によっては、未成年者向けの講座を運営する際に、年齢制限や保護者同意の確認が必要となる場合があります。\n\n\n**注意点:**\n\n*  講座の内容によっては、年齢制限や保護者同意が必要となる場合があります。\n*  未成年者向けの講座を受講する際は、必ず運営

In [ ]:
# prompt: questions、baseline_answers、rag_answersを一つのテーブルに変換して、CSVファイルを作成

import pandas as pd

# Assuming questions, baseline_answers, and rag_answers are already defined from the previous code

results = []
for i in range(len(questions)):
    results.append({
        "question": questions[i],
        "baseline_answer": baseline_answers[i],
        "rag_answer": rag_answers[i]
    })

df = pd.DataFrame(results)
df.to_csv("llm_responses.csv", index=False, encoding="utf-8")


# 3. LLM as a Judge

In [36]:
golden_answers = [
    # Q1: 退会後も研究室が受講者の情報を利用できる条件
    "受講者が退会した後であっても、在籍中に取得された情報はプライバシーポリシーを遵守する形で、本規約に従って研究室が引き続き利用することができます。",  #:contentReference[oaicite:0]{index=0}

    # Q2: 未成年者が講座を受講する条件
    "未成年者が受講を希望する場合は、事前に法定代理人の同意を得ている必要があります。",  #:contentReference[oaicite:1]{index=1}

    # Q3: 他人のSlack投稿をSNSで共有することの取り扱い
    "Slack上でのやりとりやスクリーンショットをSNSにアップすることは禁止されています。",  #:contentReference[oaicite:2]{index=2}

    # Q4: 資料や動画の再利用・再配布の可否
    "講義資料や動画のURLについて、受講後であっても無断での再利用や配布は認められていません。",  #:contentReference[oaicite:3]{index=3}

    # Q5: ノイズなどのトラブル時の対応
    "受講者のインフラに起因するノイズ等のトラブルで他の受講環境に悪影響がある場合、改善するまで参加を一時的に制限されることがあります。"  #:contentReference[oaicite:4]{index=4}
]

In [ ]:
# HTML読み込み（テキストとして評価に含める）
with open("/content/lecture-ai-engineering/day3/data/松尾・岩澤研究室 講座受講規約.html", "r", encoding="utf-8") as f:
    reference_html = f.read()

In [ ]:
template_three_criteria_with_ref = (
    "You are an expert evaluator for answers generated by an AI system.\n"
    "Below is the full official reference document that should be used to evaluate the answers.\n\n"
    "[Reference Document Start]\n{reference_text}\n[Reference Document End]\n\n"
    "Please evaluate the User Answer against this reference document and the question provided using the following criteria:\n"
    "1. Correctness (0-5): Is the User Answer factually correct based on the reference document?\n"
    "2. Completeness (0-5): Does the User Answer cover all necessary points from the reference document?\n"
    "3. Relevance (0-5): Is the User Answer directly relevant to the question?\n\n"
    "Only return your answer in this format:\n"
    "Correctness: <0-5>\nCompleteness: <0-5>\nRelevance: <0-5>\n\n"
    "### Question:\n{query}\n\n### User Answer:\n{user_answer}"
)

In [ ]:
template_strict_eval = (
    "You are an expert evaluator judging whether a User Answer is factually aligned with a given official document.\n\n"
    "The question has only one correct answer, and it must be consistent with the provided reference document.\n"
    "Please strictly evaluate the User Answer according to the three criteria below, using ONLY the reference document.\n\n"
    "If a fact or claim is not explicitly supported by the reference document, deduct points.\n"
    "Even if the User Answer sounds fluent or plausible, deduct points if it includes hallucinated or irrelevant content.\n\n"
    "Criteria:\n"
    "1. Correctness (0-5): Factually correct and consistent with the reference?\n"
    "2. Completeness (0-5): Does it fully answer the question based on the reference?\n"
    "3. Relevance (0-5): Does it directly address the question without extraneous or fabricated content?\n\n"
    "Answer format (numbers only):\n"
    "Correctness: <0-5>\nCompleteness: <0-5>\nRelevance: <0-5>\n\n"
    "### Reference Document:\n{reference_text}\n\n"
    "### Question:\n{query}\n\n"
    "### User Answer:\n{user_answer}"
)


In [ ]:
def evaluate_answer_with_reference(query, user_answer, golden_answer):
    """
    golden_answer（模範解答）を参照して user_answer の正確性・完全性・関連性を評価します。
    OpenAI GPT-4o-miniを使って3軸評価（0-5）を返します。
    """

    # 厳格な評価テンプレート（golden_answerをreferenceとして使う）
    prompt = (
        "あなたはAIによる回答の評価者です。\n\n"
        "以下は「ある質問」に対するユーザーの回答と、それに対応する模範解答（golden answer）です。\n"
        "模範解答と比較して、ユーザーの回答がどの程度正確で、完全で、質問に直接関連しているかを評価してください。\n\n"
        "評価基準は以下の通りです：\n"
        "1. 正確性（Correctness）: golden answerと矛盾なく事実に基づいているか？\n"
        "2. 完全性（Completeness）: golden answerに含まれる要点をどれだけカバーしているか？\n"
        "3. 関連性（Relevance）: 回答が質問に対して直接的か？無関係な話が混じっていないか？\n\n"
        "それぞれ0〜5点で評価し、以下の形式で返してください。\n\n"
        "Correctness: <0-5>\nCompleteness: <0-5>\nRelevance: <0-5>\n\n"
        "### 質問:\n{query}\n\n"
        "### ユーザーの回答:\n{user_answer}\n\n"
        "### 模範解答:\n{golden_answer}\n"
    ).format(query=query, user_answer=user_answer, golden_answer=golden_answer)

    try:
        response = openai_generator(prompt)
        scores = [int(s) for s in re.findall(r"\d+", response)]

        if len(scores) == 3:
            return scores
        else:
            print("スコア抽出失敗:", response)
            return [0, 0, 0]

    except Exception as e:
        print("評価エラー:", e)
        return [0, 0, 0]

In [37]:
from openai import OpenAI
from google.colab import userdata
import pandas as pd

client = OpenAI(api_key=userdata.get("OPENAI_API_KEY"), max_retries=5, timeout=60)

# OpenAI 呼び出し
def openai_generator(prompt):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content.strip()

# 評価関数：各項目ごとにスコア抽出
import re

# def evaluate_answer_with_reference(query, user_answer, reference_text):
#     prompt = template_three_criteria_with_ref.format(
#         query=query,
#         user_answer=user_answer,
#         reference_text=reference_text
#     )
#     output = openai_generator(prompt)

#     import re
#     try:
#         scores = [int(s) for s in re.findall(r"\d+", output)]
#         if len(scores) == 3:
#             return scores
#         else:
#             print("スコア抽出失敗:", output)
#             return [0, 0, 0]
#     except:
#         print("評価エラー:", output)
#         return [0, 0, 0]

# 評価実行
records = []

for i in range(len(questions)):
    row = {
        "question": questions[i],
        "baseline_answer": baseline_answers[i],
        "rag_answer": rag_answers[i]
    }

    # golden_answers[i] を参照にして採点
    row["baseline_correctness"], row["baseline_completeness"], row["baseline_relevance"] = evaluate_answer_with_reference(
        questions[i], baseline_answers[i], golden_answers[i]
    )
    row["rag_correctness"], row["rag_completeness"], row["rag_relevance"] = evaluate_answer_with_reference(
        questions[i], rag_answers[i], golden_answers[i]
    )

    records.append(row)

# DataFrameに変換・保存
df_eval = pd.DataFrame(records)
df_eval.to_csv("llm_evaluation_detailed.csv", index=False, encoding="utf-8")
df_eval


,question,baseline_answer,rag_answer,baseline_correctness,baseline_completeness,baseline_relevance,rag_correctness,rag_completeness,rag_relevance
0,松尾・岩澤研究室の講座において、退会した後も研究室が受講者の情報を利用できる条件とは何ですか？,松尾・岩澤研究室の講座において、退会後も研究室が受講者の情報を利用できる条件は、**研究室...,松尾・岩澤研究室の講座において、退会した後も研究室が受講者の情報を利用できる条件は、**「本...,2,2,3,4,3,5
1,未成年者が講座を受講するには、どのような条件が求められますか？,未成年者が講座を受講するには、以下の条件が求められます。\n\n**1. 年齢制限:**\...,未成年者が講座を受講するには、以下の条件が求められます。 \n\n* **未成年者の本人によ...,5,3,5,5,5,5
2,受講者が他の人のSlack投稿をSNSで共有することは、講座規約上どう扱われますか？,講座規約上、受講者が他の人のSlack投稿をSNSで共有することは、**多様で、明確な回答は...,講座規約に記載されている通り、**受講者が他の人のSlack投稿をSNSで共有することは、個...,1,1,3,1,2,3
3,講座で使用された資料や動画を、受講後に再利用・再配布することは許可されていますか？,講座で使用された資料や動画の再利用・再配布については、**講座の主催者や提供者によって異なり...,いいえ、講座で使用された資料や動画を、受講後に再利用・再配布することは許可されていません。 ...,0,0,0,5,5,5
4,講義中にノイズなどのトラブルを発生させた場合、講座への参加はどうなりますか？,講義中にノイズなどのトラブルが発生した場合、講座への参加は**状況によって異なります**。\...,講義中にノイズなどのトラブルが発生した場合、当研究室の判断により、受講環境が改善するまでの間...,1,1,2,5,5,5


In [38]:
def evaluate_answer_with_reference(query, user_answer, golden_answer):
    """
    golden_answerを基準に user_answer を評価し、各項目のスコアとその理由（説明）を返す。
    戻り値: (correctness, completeness, relevance, explanation)
    """

    prompt = (
        "あなたはAIによる回答の評価者です。\n\n"
        "以下はある質問に対するユーザーの回答と、その模範解答（golden answer）です。\n"
        "以下の3つの観点でユーザー回答を0〜5点で評価し、それぞれについて簡単な理由を説明してください。\n\n"
        "【観点】\n"
        "1. 正確性（Correctness）: golden answerと矛盾せず、事実として正しいか？\n"
        "2. 完全性（Completeness）: golden answerに含まれる要点がカバーされているか？\n"
        "3. 関連性（Relevance）: 回答が質問に直接関連し、無関係な内容を含んでいないか？\n\n"
        "【出力形式】（必ず以下の形式で）\n"
        "Correctness: <スコア>（理由）\n"
        "Completeness: <スコア>（理由）\n"
        "Relevance: <スコア>（理由）\n\n"
        "### 質問:\n{query}\n\n"
        "### ユーザーの回答:\n{user_answer}\n\n"
        "### 模範解答:\n{golden_answer}\n"
    ).format(query=query, user_answer=user_answer, golden_answer=golden_answer)

    try:
        response = openai_generator(prompt)

        # スコアの抽出
        scores = [int(s) for s in re.findall(r"(?<=: )\d", response)]
        explanations = {}
        for line in response.strip().split("\n"):
            if line.startswith("Correctness"):
                explanations["correctness_reason"] = line
            elif line.startswith("Completeness"):
                explanations["completeness_reason"] = line
            elif line.startswith("Relevance"):
                explanations["relevance_reason"] = line

        if len(scores) == 3:
            return (
                scores[0], scores[1], scores[2],
                explanations.get("correctness_reason", ""),
                explanations.get("completeness_reason", ""),
                explanations.get("relevance_reason", "")
            )
        else:
            print("スコア抽出失敗:", response)
            return 0, 0, 0, "N/A", "N/A", "N/A"

    except Exception as e:
        print("評価エラー:", e)
        return 0, 0, 0, "N/A", "N/A", "N/A"


In [39]:
records = []

for i in range(len(questions)):
    row = {
        "question": questions[i],
        "baseline_answer": baseline_answers[i],
        "rag_answer": rag_answers[i],
        "golden_answer": golden_answers[i]
    }

    # baseline
    b_corr, b_comp, b_rel, b_exp_corr, b_exp_comp, b_exp_rel = evaluate_answer_with_reference(
        questions[i], baseline_answers[i], golden_answers[i]
    )
    row["baseline_correctness"] = b_corr
    row["baseline_completeness"] = b_comp
    row["baseline_relevance"] = b_rel
    row["baseline_correctness_explanation"] = b_exp_corr
    row["baseline_completeness_explanation"] = b_exp_comp
    row["baseline_relevance_explanation"] = b_exp_rel

    # rag
    r_corr, r_comp, r_rel, r_exp_corr, r_exp_comp, r_exp_rel = evaluate_answer_with_reference(
        questions[i], rag_answers[i], golden_answers[i]
    )
    row["rag_correctness"] = r_corr
    row["rag_completeness"] = r_comp
    row["rag_relevance"] = r_rel
    row["rag_correctness_explanation"] = r_exp_corr
    row["rag_completeness_explanation"] = r_exp_comp
    row["rag_relevance_explanation"] = r_exp_rel

    records.append(row)


In [43]:
# DataFrame に変換
df_eval = pd.DataFrame(records)

,question,baseline_answer,rag_answer,golden_answer,baseline_correctness,baseline_completeness,baseline_relevance,baseline_correctness_explanation,baseline_completeness_explanation,baseline_relevance_explanation,rag_correctness,rag_completeness,rag_relevance,rag_correctness_explanation,rag_completeness_explanation,rag_relevance_explanation
0,松尾・岩澤研究室の講座において、退会した後も研究室が受講者の情報を利用できる条件とは何ですか？,松尾・岩澤研究室の講座において、退会後も研究室が受講者の情報を利用できる条件は、**研究室...,松尾・岩澤研究室の講座において、退会した後も研究室が受講者の情報を利用できる条件は、**「本...,受講者が退会した後であっても、在籍中に取得された情報はプライバシーポリシーを遵守する形で、本...,3,2,4,Correctness: 3（ユーザーの回答は、一般的な条件を述べており、一部正しいですが、...,Completeness: 2（ユーザーの回答は、受講者の情報利用についての一般的な条件を述...,Relevance: 4（ユーザーの回答は質問に関連した内容を提供しているが、具体的に研究室...,5,3,5,Correctness: 5（ユーザーの回答は事実として正しく、プライバシーポリシーに従うこ...,Completeness: 3（ユーザーの回答は基本的な条件を示していますが、在籍中に取得さ...,Relevance: 5（ユーザーの回答は質問に直接関連しており、無関係な内容は含まれていな...
1,未成年者が講座を受講するには、どのような条件が求められますか？,未成年者が講座を受講するには、以下の条件が求められます。\n\n**1. 年齢制限:**\...,未成年者が講座を受講するには、以下の条件が求められます。 \n\n* **未成年者の本人によ...,未成年者が受講を希望する場合は、事前に法定代理人の同意を得ている必要があります。,4,3,5,Correctness: 4（ユーザーの回答は、未成年者が講座を受講する際に保護者または親権...,Completeness: 3（ユーザーの回答にはさまざまな条件が列挙されており、具体的な講...,Relevance: 5（ユーザーの回答は、未成年者の受講条件に関連した情報を提供しており、...,5,4,5,Correctness: 5（ユーザーの回答は、未成年者が講座を受講する際に法定代理人の同意...,Completeness: 4（ユーザーの回答は、「法定代理人の同意」という重要な要点を含ん...,Relevance: 5（ユーザーの回答は、質問に対して直接関連しており、無関係な内容が含ま...
2,受講者が他の人のSlack投稿をSNSで共有することは、講座規約上どう扱われますか？,講座規約上、受講者が他の人のSlack投稿をSNSで共有することは、**多様で、明確な回答は...,講座規約に記載されている通り、**受講者が他の人のSlack投稿をSNSで共有することは、個...,Slack上でのやりとりやスクリーンショットをSNSにアップすることは禁止されています。,2,2,3,Correctness: 2（ユーザーの回答は、正確性としては講座規約に対する具体的なルール...,Completeness: 2（ユーザーの回答は多くの要点に言及しているが、最も重要な「Sl...,Relevance: 3（ユーザーの回答は質問に関連しているが、一般論や考慮点に重きを置きす...,1,1,2,Correctness: 1（ユーザーの回答は、模範解答の具体的な禁止事項と矛盾しているため...,Completeness: 1（ユーザーの回答は、模範解答の主旨である「禁止」という重要な点...,Relevance: 2（ユーザーの回答は質問に関連性がある部分もあるが、核心に触れられてい...
3,講座で使用された資料や動画を、受講後に再利用・再配布することは許可されていますか？,講座で使用された資料や動画の再利用・再配布については、**講座の主催者や提供者によって異なり...,いいえ、講座で使用された資料や動画を、受講後に再利用・再配布することは許可されていません。 ...,講義資料や動画のURLについて、受講後であっても無断での再利用や配布は認められていません。,2,2,4,Correctness: 2（ユーザーの回答は一般論として正しい面がありますが、具体的に無断...,Completeness: 2（ユーザーの回答は再利用や再配布が許可される場合の条件をいくつ...,Relevance: 4（ユーザーの回答はテーマに関連しており、一般的な条件や例を挙げていま...,5,4,5,Correctness: 5（ユーザーの回答は講座の規約に基づいており、正確です。具体的には...,Completeness: 4（ユーザーの回答は、一般的な内容と規約の具体的な引用を含んでい...,Relevance: 5（質問について直接的に答えており、資料や動画の再利用・再配布に関する...
4,講義中にノイズなどのトラブルを発生させた場合、講座への参加はどうなりますか？,講義中にノイズなどのトラブルが発生した場合、講座への参加は**状況によって異なります**。\...,講義中にノイズなどのトラブルが発生した場合、当研究室の判断により、受講環境が改善するまでの間...,受講者のインフラに起因するノイズ等のトラブルで他の受講環境に悪影響がある場合、改善するまで参...,3,3,4,Correctness: 3（ユーザーの回答は、トラブルが発生した場合の参加に関する状況を概...,Completeness: 3（ユーザーの回答は様々なケースや対応を考慮しているが、模範解答...,Relevance: 4（ユーザーの回答は講座への参加に関連する内容を扱っており、講義中のト...,5,4,5,Correctness: 5（ユーザーの回答は、模範解答と矛盾せず、事実として正しい内容です...,Completeness: 4（ユーザーの回答は、模範解答の要点をほぼカバーしていますが、「...,Relevance: 5（回答は質問に直接関連しており、無関係な内容は含まれていません。講義...


In [51]:
# DataFrame に変換
df_eval = pd.DataFrame(records)

In [52]:
score_columns = [
    "baseline_correctness", "baseline_completeness", "baseline_relevance",
    "rag_correctness", "rag_completeness", "rag_relevance"
]

# スコア列のみ抽出して表示
df_eval[score_columns]

,baseline_correctness,baseline_completeness,baseline_relevance,rag_correctness,rag_completeness,rag_relevance
0,3,2,4,5,3,5
1,4,3,5,5,4,5
2,2,2,3,1,1,2
3,2,2,4,5,4,5
4,3,3,4,5,4,5


In [49]:


# スコア列（数値）のみ削除
score_columns = [
    "baseline_correctness", "baseline_completeness", "baseline_relevance",
    "rag_correctness", "rag_completeness", "rag_relevance"
]
df_eval = df_eval.drop(columns=score_columns, errors="ignore")

# CSV に保存
df_eval.to_csv("llm_evaluation_detailed.csv", index=False, encoding="utf-8-sig")
df_eval

,question,baseline_answer,rag_answer,golden_answer,baseline_correctness_explanation,baseline_completeness_explanation,baseline_relevance_explanation,rag_correctness_explanation,rag_completeness_explanation,rag_relevance_explanation
0,松尾・岩澤研究室の講座において、退会した後も研究室が受講者の情報を利用できる条件とは何ですか？,松尾・岩澤研究室の講座において、退会後も研究室が受講者の情報を利用できる条件は、**研究室...,松尾・岩澤研究室の講座において、退会した後も研究室が受講者の情報を利用できる条件は、**「本...,受講者が退会した後であっても、在籍中に取得された情報はプライバシーポリシーを遵守する形で、本...,Correctness: 3（ユーザーの回答は、一般的な条件を述べており、一部正しいですが、...,Completeness: 2（ユーザーの回答は、受講者の情報利用についての一般的な条件を述...,Relevance: 4（ユーザーの回答は質問に関連した内容を提供しているが、具体的に研究室...,Correctness: 5（ユーザーの回答は事実として正しく、プライバシーポリシーに従うこ...,Completeness: 3（ユーザーの回答は基本的な条件を示していますが、在籍中に取得さ...,Relevance: 5（ユーザーの回答は質問に直接関連しており、無関係な内容は含まれていな...
1,未成年者が講座を受講するには、どのような条件が求められますか？,未成年者が講座を受講するには、以下の条件が求められます。\n\n**1. 年齢制限:**\...,未成年者が講座を受講するには、以下の条件が求められます。 \n\n* **未成年者の本人によ...,未成年者が受講を希望する場合は、事前に法定代理人の同意を得ている必要があります。,Correctness: 4（ユーザーの回答は、未成年者が講座を受講する際に保護者または親権...,Completeness: 3（ユーザーの回答にはさまざまな条件が列挙されており、具体的な講...,Relevance: 5（ユーザーの回答は、未成年者の受講条件に関連した情報を提供しており、...,Correctness: 5（ユーザーの回答は、未成年者が講座を受講する際に法定代理人の同意...,Completeness: 4（ユーザーの回答は、「法定代理人の同意」という重要な要点を含ん...,Relevance: 5（ユーザーの回答は、質問に対して直接関連しており、無関係な内容が含ま...
2,受講者が他の人のSlack投稿をSNSで共有することは、講座規約上どう扱われますか？,講座規約上、受講者が他の人のSlack投稿をSNSで共有することは、**多様で、明確な回答は...,講座規約に記載されている通り、**受講者が他の人のSlack投稿をSNSで共有することは、個...,Slack上でのやりとりやスクリーンショットをSNSにアップすることは禁止されています。,Correctness: 2（ユーザーの回答は、正確性としては講座規約に対する具体的なルール...,Completeness: 2（ユーザーの回答は多くの要点に言及しているが、最も重要な「Sl...,Relevance: 3（ユーザーの回答は質問に関連しているが、一般論や考慮点に重きを置きす...,Correctness: 1（ユーザーの回答は、模範解答の具体的な禁止事項と矛盾しているため...,Completeness: 1（ユーザーの回答は、模範解答の主旨である「禁止」という重要な点...,Relevance: 2（ユーザーの回答は質問に関連性がある部分もあるが、核心に触れられてい...
3,講座で使用された資料や動画を、受講後に再利用・再配布することは許可されていますか？,講座で使用された資料や動画の再利用・再配布については、**講座の主催者や提供者によって異なり...,いいえ、講座で使用された資料や動画を、受講後に再利用・再配布することは許可されていません。 ...,講義資料や動画のURLについて、受講後であっても無断での再利用や配布は認められていません。,Correctness: 2（ユーザーの回答は一般論として正しい面がありますが、具体的に無断...,Completeness: 2（ユーザーの回答は再利用や再配布が許可される場合の条件をいくつ...,Relevance: 4（ユーザーの回答はテーマに関連しており、一般的な条件や例を挙げていま...,Correctness: 5（ユーザーの回答は講座の規約に基づいており、正確です。具体的には...,Completeness: 4（ユーザーの回答は、一般的な内容と規約の具体的な引用を含んでい...,Relevance: 5（質問について直接的に答えており、資料や動画の再利用・再配布に関する...
4,講義中にノイズなどのトラブルを発生させた場合、講座への参加はどうなりますか？,講義中にノイズなどのトラブルが発生した場合、講座への参加は**状況によって異なります**。\...,講義中にノイズなどのトラブルが発生した場合、当研究室の判断により、受講環境が改善するまでの間...,受講者のインフラに起因するノイズ等のトラブルで他の受講環境に悪影響がある場合、改善するまで参...,Correctness: 3（ユーザーの回答は、トラブルが発生した場合の参加に関する状況を概...,Completeness: 3（ユーザーの回答は様々なケースや対応を考慮しているが、模範解答...,Relevance: 4（ユーザーの回答は講座への参加に関連する内容を扱っており、講義中のト...,Correctness: 5（ユーザーの回答は、模範解答と矛盾せず、事実として正しい内容です...,Completeness: 4（ユーザーの回答は、模範解答の要点をほぼカバーしていますが、「...,Relevance: 5（回答は質問に直接関連しており、無関係な内容は含まれていません。講義...


# 4. RAGのデバッグ

In [56]:
def debug_rag_chunks_for_question(index, questions, documents, doc_embeddings, emb_model, top_k=5):
    """
    質問に対して、類似度の高いチャンクとそのスコアを表示する（FAISS非使用）。

    Parameters:
        index (int): questionsのインデックス
        questions (list of str): 質問リスト
        documents (list of str): 事前に分割された文書チャンク群
        doc_embeddings (Tensor): documentsに対応する埋め込みベクトル（convert_to_tensor=Trueで生成）
        emb_model: SentenceTransformerなどのエンコーダー
        top_k (int): 表示する上位チャンク数
    """
    question = questions[index]
    print(f"\n=== [質問 {index}] ===\n{question}\n")

    # 質問の埋め込みベクトルを生成
    q_embedding = emb_model.encode([question], convert_to_tensor=True)

    # 類似度スコア（内積）を計算
    scores = torch.matmul(q_embedding, doc_embeddings.T)[0].cpu().numpy()

    # 上位 top_k のインデックスとスコア
    top_indices = scores.argsort()[::-1][:top_k]

    print(f"--- 類似チャンク（Top {top_k}） ---")
    for rank, i in enumerate(top_indices):
        print(f"[{rank+1}] 類似度スコア: {scores[i]:.4f}")
        print(f"内容: {documents[i][:300]}...")
        print("-" * 40)

In [57]:
debug_rag_chunks_for_question(
    index=2,  # questions[2]
    questions=questions,
    documents=documents,
    doc_embeddings=doc_embeddings,
    emb_model=emb_model,
    top_k=5
)


=== [質問 2] ===
受講者が他の人のSlack投稿をSNSで共有することは、講座規約上どう扱われますか？

--- 類似チャンク（Top 5） ---
[1] 類似度スコア: 0.8818
内容: 2. なお、本規約より受講者は受講者からの質問や個人情報に該当しない部分については研究室がデータの利活用する事を了承するものとします...
----------------------------------------
[2] 類似度スコア: 0.8758
内容: 第1条（適用範囲）
1. 本規約は、講座を受講するすべての対象者(以下、受講者)と当研究室との間における一切の関係に適用されるものとします...
----------------------------------------
[3] 類似度スコア: 0.8706
内容: 2. 受講者が各講座の受講を修了していたとしても受講者登録が行われている場合、当研究室から各受講者に新たな講座案内等の情報発信を行うことがあります...
----------------------------------------
[4] 類似度スコア: 0.8672
内容: 第6条（譲渡の禁止）
1. 受講者は、受講者としての資格、地位、権利または義務につき、第三者に移転もしくは譲渡し、または担保に供し、その他一切の処分をすることはできません...
----------------------------------------
[5] 類似度スコア: 0.8629
内容: 第3条（受講申込）
1. 受講希望者は本規約に同意の上、当研究室の定める方法によって受講申込を行い、当研究室がこれを承認することによって完了するものとします...
----------------------------------------


In [34]:
def identify_factual_errors(query, user_answer, reference_text):
    prompt = (
        "You are a strict factual checker. Given a reference document, a question, and a user answer, "
        "identify any factual inaccuracies in the user answer based on the reference.\n\n"
        "If the user answer contains claims not supported by the reference, or contradicts it, list them.（日本語で）\n"
        "If there are no factual errors, simply say: None.\n\n"
        "### Reference Document:\n{reference_text}\n\n"
        "### Question:\n{query}\n\n"
        "### User Answer:\n{user_answer}\n\n"
        "Factual Errors:"
    ).format(reference_text=reference_text, query=query, user_answer=user_answer)

    return openai_generator(prompt)


In [ ]:
def compare_baseline_vs_rag(query, baseline_answer, rag_answer, reference_text):
    prompt = (
        "You are an impartial evaluator. Based ONLY on the reference document, determine which answer is more accurate and appropriate for the given question.\n\n"
        "Choose the better one strictly based on factual correctness, completeness, and relevance to the question.\n"
        "If both answers are equally good, say: Equal\n"
        "Choose from: Baseline / RAG / Equal\n\n"
        "### Reference Document:\n{reference_text}\n\n"
        "### Question:\n{query}\n\n"
        "### Baseline Answer:\n{baseline_answer}\n\n"
        "### RAG Answer:\n{rag_answer}\n\n"
        "Better Answer:".format(
            reference_text=reference_text,
            query=query,
            baseline_answer=baseline_answer,
            rag_answer=rag_answer
        )
    )

    return openai_generator(prompt)


In [35]:
records = []

for i in range(len(questions)):
    question = questions[i]
    base_ans = baseline_answers[i]
    rag_ans = rag_answers[i]

    # ベースラインとRAGの事実誤認指摘
    baseline_errors = identify_factual_errors(question, base_ans, reference_html)
    rag_errors = identify_factual_errors(question, rag_ans, reference_html)

    # 相対評価
    comparison_result = compare_baseline_vs_rag(question, base_ans, rag_ans, reference_html)

    records.append({
        "question": question,
        "baseline_answer": base_ans,
        "rag_answer": rag_ans,
        "baseline_errors": baseline_errors,
        "rag_errors": rag_errors,
        "better_answer": comparison_result.strip()
    })

df_compare = pd.DataFrame(records)
df_compare.to_csv("rag_vs_baseline_comparison.csv", index=False, encoding="utf-8")
df_compare


,question,baseline_answer,rag_answer,baseline_errors,rag_errors,better_answer
0,松尾・岩澤研究室の講座において、退会した後も研究室が受講者の情報を利用できる条件とは何ですか？,松尾・岩澤研究室の講座において、退会後も研究室が受講者の情報を利用できる条件は、**研究室...,松尾・岩澤研究室の講座において、退会した後も研究室が受講者の情報を利用できる条件は、**「本...,以下の事実誤認を確認しました。\n\n1. **研究室のポリシーや契約内容によって異なるとい...,- ユーザーの回答は、退会後に研究室が受講者の情報を利用できる条件について「本プライバシーポ...,RAG
1,未成年者が講座を受講するには、どのような条件が求められますか？,未成年者が講座を受講するには、以下の条件が求められます。\n\n**1. 年齢制限:**\...,未成年者が講座を受講するには、以下の条件が求められます。 \n\n* **未成年者の本人によ...,以下の誤りがあります：\n\n1. 「一般的には18歳未満の未成年者向け講座は、保護者または...,以下の事実誤認があります。\n\n1. **未成年者の本人による申込み:** ユーザーが主張...,RAG
2,受講者が他の人のSlack投稿をSNSで共有することは、講座規約上どう扱われますか？,講座規約上、受講者が他の人のSlack投稿をSNSで共有することは、**多様で、明確な回答は...,講座規約に記載されている通り、**受講者が他の人のSlack投稿をSNSで共有することは、個...,以下の事実誤認があります。\n\n1. **契約違反の具体的な表現:** ユーザーの回答では...,1. ユーザーの回答には「**研究室がデータの利活用する事を了承するものとします**」という...,RAG
3,講座で使用された資料や動画を、受講後に再利用・再配布することは許可されていますか？,講座で使用された資料や動画の再利用・再配布については、**講座の主催者や提供者によって異なり...,いいえ、講座で使用された資料や動画を、受講後に再利用・再配布することは許可されていません。 ...,- ユーザーの回答には、「講座の主催者や提供者によって異なります」と記載されていますが、参考...,None.,RAG
4,講義中にノイズなどのトラブルを発生させた場合、講座への参加はどうなりますか？,講義中にノイズなどのトラブルが発生した場合、講座への参加は**状況によって異なります**。\...,講義中にノイズなどのトラブルが発生した場合、当研究室の判断により、受講環境が改善するまでの間...,1. **講座主催者による判断の部分**: ユーザーの答えでは、「状況によっては、参加を中止...,該当する事実誤りはありません。,RAG
